In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import wandb
import pdb
from adjustText import adjust_text
import pandas as pd
import os

In [ ]:
def get_val_metrics(runs, index=0, exclude_keyword='ignore'):
    run_f1_scores = {}
    run_auroc_val_scores = {}
    
    for run in runs:
        # print(run.name)
        if exclude_keyword.lower() not in run.name.lower():

            history = run.history(pandas=True)
            # pdb.set_trace()
            # print(run.id)
            # pdb.set_trace()
            val_f1_scores = history['f1_val'].dropna().tolist()
            auroc_val_scores = history['auroc_val'].dropna().tolist()
            
            run_f1_scores[run.id] = val_f1_scores
            run_auroc_val_scores[run.id] = auroc_val_scores
    
    # mean_auroc_scores = [f1_scores[index] for f1_scores in run_f1_scores.values()]
    # mean_auroc_val_scores = [auroc_val_scores[index] for auroc_val_scores in run_auroc_val_scores.values()]
    
    # overall_mean_f1 = np.mean(mean_f1_scores)
    # overall_std_f1 = np.std(mean_f1_scores)
    
    # overall_mean_auroc_val = np.mean(mean_auroc_val_scores)
    # overall_std_auroc_val = np.std(mean_auroc_val_scores)
    
    # print(f"Mean F1 score for the {index}th epoch across all runs: {overall_mean_f1:.4f}")
    # print(f"Std. F1 score for the {index}th epoch across all runs: {overall_std_f1:.4f}")
    
    # print(f"Mean AUROC VAL score for the {index}th epoch across all runs: {overall_mean_auroc_val:.4f}")
    # print(f"Std. AUROC VAL score for the {index}th epoch across all runs: {overall_std_auroc_val:.4f}")
    return list(run_f1_scores.values()), list(run_auroc_val_scores.values())

def get_val_metrics_scan(runs, index=0, exclude_keyword='ignore'):
    run_f1_scores = {}
    run_auroc_val_scores = {}
    
    for run in runs:
        if exclude_keyword.lower() not in run.name.lower():
            # Use scan_history() instead of history()
            history_data = []
            for row in run.scan_history(keys=["f1_val", "auroc_val"]):
                history_data.append(row)
            
            # Convert to DataFrame
            history = pd.DataFrame(history_data)
            
            val_f1_scores = history['f1_val'].dropna().tolist()
            auroc_val_scores = history['auroc_val'].dropna().tolist()
            
            run_f1_scores[run.id] = val_f1_scores
            run_auroc_val_scores[run.id] = auroc_val_scores
    
    return list(run_f1_scores.values()), list(run_auroc_val_scores.values())

def get_mean_std_per_agg(scores):
    means = [np.mean([sublist[i] for sublist in scores]) for i in range(len(scores[0]))]
    stds = [np.std([sublist[i] for sublist in scores]) for i in range(len(scores[0]))]
    return np.max(means), stds[np.argmax(means)], np.argmax(means)

def get_max(scores):
    max_per_fold = [np.max(fold) for fold in scores]
    max_epochs = [np.argmax(fold) for fold in scores]
    return max_per_fold, max_epochs

MDL_COLOR = {
    'resnet50':'r',
    'uni':'b',
    'gigapath': 'g'
}

def plot_results(uni_res, resnet_res, gigapath_res, title, colors=MDL_COLOR):
    # Create the x-axis values
    

    # Create the figure and axis
    fig, ax = plt.subplots(figsize=(15, 10), dpi=150)

    resnet50_means, resnet50_stds = resnet_res
    uni_means, uni_stds = uni_res
    gigapath_means, gigapath_stds = gigapath_res
    
    x = range(0, len(resnet50_means))
    # Plot the mean with error bars
    ax.errorbar(x, resnet50_means, yerr=resnet50_stds, fmt='o', capsize=5, color=colors['resnet50'])
    ax.errorbar(x, uni_means, yerr=uni_stds, fmt='x', capsize=5, color=colors['uni'])
    ax.errorbar(x, gigapath_means, yerr=gigapath_stds, fmt='s', capsize=5, color=colors['gigapath'])

    # Connect the mean values with lines
    ax.plot(x, resnet50_means, linestyle='-', color=colors['resnet50'], label='resnet50')
    ax.plot(x, uni_means, linestyle='--', color=colors['uni'], label='uni')
    ax.plot(x, gigapath_means, linestyle='-.', color=colors['gigapath'], label='gigapath')
    # Create text annotations
    texts = []
    # for i in x:
    #     resnet_label = f'{resnet50_means[i]:.2f} ± {resnet50_stds[i]:.2f}'
    #     uni_label = f'{uni_means[i]:.2f} ± {uni_stds[i]:.2f}'
    #     gigapath_label = f'{gigapath_means[i]:.2f} ± {gigapath_stds[i]:.2f}'
        
    #     texts.append(ax.text(i, resnet50_means[i], resnet_label, color='r', ha='center', va='bottom'))
    #     texts.append(ax.text(i, uni_means[i], uni_label, color='b', ha='center', va='bottom'))
    #     texts.append(ax.text(i, gigapath_means[i], gigapath_label, color='g', ha='center', va='bottom'))
    # Identify significant points to annotate
    significant_indices = [0]  # First and last points

    # Find the highest point for each array
    highest_resnet50_index = np.argmax(resnet50_means) # max(range(len(resnet50_means)), key=lambda i: resnet50_means[i])
    highest_uni_index = np.argmax(uni_means) # max(range(len(uni_means)), key=lambda i: uni_means[i])
    highest_gigapath_index = np.argmax(gigapath_means) # max(range(len(gigapath_means)), key=lambda i: gigapath_means[i])
    # pdb.set_trace()
    # # Add the highest indices to the significant indices
    # significant_indices.extend([highest_resnet50_index, highest_uni_index, highest_gigapath_index])

    # Create text annotations for significant points only
    texts = []
    for i in significant_indices:
        resnet_label = f'{resnet50_means[i]:.2f} ± {resnet50_stds[i]:.2f}'
        uni_label = f'{uni_means[i]:.2f} ± {uni_stds[i]:.2f}'
        gigapath_label = f'{gigapath_means[i]:.2f} ± {gigapath_stds[i]:.2f}'
        
        texts.append(ax.text(i, resnet50_means[i], resnet_label, color=colors['resnet50'], ha='center', va='bottom', fontsize=13))
        texts.append(ax.text(i, uni_means[i], uni_label, color=colors['uni'], ha='center', va='bottom', fontsize=13))
        texts.append(ax.text(i, gigapath_means[i], gigapath_label, color=colors['gigapath'], ha='center', va='bottom', fontsize=13))
    
    ### plot highest points
    texts.append(ax.text(highest_resnet50_index, resnet50_means[highest_resnet50_index],  f'{resnet50_means[highest_resnet50_index]:.2f} ± {resnet50_stds[highest_resnet50_index]:.2f}', color=colors['resnet50'], ha='center', va='bottom', fontweight='bold', fontsize=13))
    texts.append(ax.text(highest_uni_index, uni_means[highest_uni_index], f'{uni_means[highest_uni_index]:.2f} ± {uni_stds[highest_uni_index]:.2f}', color=colors['uni'], ha='center', va='bottom', fontweight='bold', fontsize=13))
    texts.append(ax.text(highest_gigapath_index, gigapath_means[highest_gigapath_index], f'{gigapath_means[highest_gigapath_index]:.2f} ± {gigapath_stds[highest_gigapath_index]:.2f}', color=colors['gigapath'], ha='center', va='bottom', fontweight='bold', fontsize=13))


    adjust_text(texts, arrowprops=dict(arrowstyle='->', color='k', lw=1.5))
    # Add labels for the mean values
    ax.legend()

    # Set the axis labels and title
    ax.set_xlabel('Epoch', fontsize=16)
    ax.set_ylabel('Avg. cross-validation score', fontsize=16)
    ax.set_title(f'{title}', fontsize=20)

    # Set the x-axis ticks
    ax.set_xticks(x)
    plt.xticks(fontsize=12)              # Increase x-tick label font size
    plt.yticks(fontsize=12)     
    ax.grid(True, linestyle='--', alpha=0.85)
    # Show the plot
    plt.show()

def plot_results_no_gigapath(uni_res, resnet_res, gigapath_res, title, colors=MDL_COLOR):
    # Create the x-axis values
    

    # Create the figure and axis
    fig, ax = plt.subplots(figsize=(15, 10), dpi=150)

    resnet50_means, resnet50_stds = resnet_res
    uni_means, uni_stds = uni_res
    gigapath_means, gigapath_stds = gigapath_res
    
    x = range(0, len(resnet50_means))
    # Plot the mean with error bars
    ax.errorbar(x, resnet50_means, yerr=resnet50_stds, fmt='o', capsize=5, color=colors['resnet50'])
    ax.errorbar(x, uni_means, yerr=uni_stds, fmt='x', capsize=5, color=colors['uni'])
    # ax.errorbar(x, gigapath_means, yerr=gigapath_stds, fmt='s', capsize=5, color=colors['gigapath'])

    # Connect the mean values with lines
    ax.plot(x, resnet50_means, linestyle='-', color=colors['resnet50'], label='resnet50')
    ax.plot(x, uni_means, linestyle='--', color=colors['uni'], label='uni')
    # ax.plot(x, gigapath_means, linestyle='-.', color=colors['gigapath'], label='gigapath')
    # Create text annotations
    texts = []
    
    significant_indices = [0]  # First and last points

    # Find the highest point for each array
    highest_resnet50_index = np.argmax(resnet50_means) # max(range(len(resnet50_means)), key=lambda i: resnet50_means[i])
    highest_uni_index = np.argmax(uni_means) # max(range(len(uni_means)), key=lambda i: uni_means[i])
    # highest_gigapath_index = np.argmax(gigapath_means) # max(range(len(gigapath_means)), key=lambda i: gigapath_means[i])
    # pdb.set_trace()
    # # Add the highest indices to the significant indices
    # significant_indices.extend([highest_resnet50_index, highest_uni_index, highest_gigapath_index])

    # Create text annotations for significant points only
    texts = []
    for i in significant_indices:
        resnet_label = f'{resnet50_means[i]:.2f} ± {resnet50_stds[i]:.2f}'
        uni_label = f'{uni_means[i]:.2f} ± {uni_stds[i]:.2f}'
        # gigapath_label = f'{gigapath_means[i]:.2f} ± {gigapath_stds[i]:.2f}'
        
        texts.append(ax.text(i, resnet50_means[i], resnet_label, color=colors['resnet50'], ha='center', va='bottom', fontsize=13))
        texts.append(ax.text(i, uni_means[i], uni_label, color=colors['uni'], ha='center', va='bottom', fontsize=13))
        # texts.append(ax.text(i, gigapath_means[i], gigapath_label, color=colors['gigapath'], ha='center', va='bottom', fontsize=13))
    
    ### plot highest points
    texts.append(ax.text(highest_resnet50_index, resnet50_means[highest_resnet50_index],  f'{resnet50_means[highest_resnet50_index]:.2f} ± {resnet50_stds[highest_resnet50_index]:.2f}', color=colors['resnet50'], ha='center', va='bottom', fontweight='bold', fontsize=13))
    texts.append(ax.text(highest_uni_index, uni_means[highest_uni_index], f'{uni_means[highest_uni_index]:.2f} ± {uni_stds[highest_uni_index]:.2f}', color=colors['uni'], ha='center', va='bottom', fontweight='bold', fontsize=13))
    # texts.append(ax.text(highest_gigapath_index, gigapath_means[highest_gigapath_index], f'{gigapath_means[highest_gigapath_index]:.2f} ± {gigapath_stds[highest_gigapath_index]:.2f}', color=colors['gigapath'], ha='center', va='bottom', fontweight='bold', fontsize=13))


    adjust_text(texts, arrowprops=dict(arrowstyle='->', color='k', lw=1.5))
    # Add labels for the mean values
    ax.legend()

    # Set the axis labels and title
    ax.set_xlabel('Epoch', fontsize=16)
    ax.set_ylabel('Avg. cross-validation score', fontsize=16)
    ax.set_title(f'{title}', fontsize=20)

    # Set the x-axis ticks
    ax.set_xticks(x)
    plt.xticks(fontsize=12)              # Increase x-tick label font size
    plt.yticks(fontsize=12)     
    ax.grid(True, linestyle='--', alpha=0.85)
    # Show the plot
    plt.show()

def plot_results_only_uni(uni_res, resnet_res, gigapath_res, title, colors=MDL_COLOR):
    # Create the x-axis values
    

    # Create the figure and axis
    fig, ax = plt.subplots(figsize=(15, 10), dpi=150)

    resnet50_means, resnet50_stds = resnet_res
    uni_means, uni_stds = uni_res
    gigapath_means, gigapath_stds = gigapath_res
    
    x = range(0, len(resnet50_means))
    # Plot the mean with error bars
    # ax.errorbar(x, resnet50_means, yerr=resnet50_stds, fmt='o', capsize=5, color=colors['resnet50'])
    ax.errorbar(x, uni_means, yerr=uni_stds, fmt='x', capsize=5, color=colors['uni'])
    # ax.errorbar(x, gigapath_means, yerr=gigapath_stds, fmt='s', capsize=5, color=colors['gigapath'])

    # Connect the mean values with lines
    # ax.plot(x, resnet50_means, linestyle='-', color=colors['resnet50'], label='resnet50')
    ax.plot(x, uni_means, linestyle='--', color=colors['uni'], label='uni')
    # ax.plot(x, gigapath_means, linestyle='-.', color=colors['gigapath'], label='gigapath')
    # Create text annotations
    texts = []
    # for i in x:
    #     resnet_label = f'{resnet50_means[i]:.2f} ± {resnet50_stds[i]:.2f}'
    #     uni_label = f'{uni_means[i]:.2f} ± {uni_stds[i]:.2f}'
    #     gigapath_label = f'{gigapath_means[i]:.2f} ± {gigapath_stds[i]:.2f}'
        
    #     texts.append(ax.text(i, resnet50_means[i], resnet_label, color='r', ha='center', va='bottom'))
    #     texts.append(ax.text(i, uni_means[i], uni_label, color='b', ha='center', va='bottom'))
    #     texts.append(ax.text(i, gigapath_means[i], gigapath_label, color='g', ha='center', va='bottom'))
    # Identify significant points to annotate
    significant_indices = [0]  # First and last points

    # Find the highest point for each array
    # highest_resnet50_index = np.argmax(resnet50_means) # max(range(len(resnet50_means)), key=lambda i: resnet50_means[i])
    highest_uni_index = np.argmax(uni_means) # max(range(len(uni_means)), key=lambda i: uni_means[i])
    # highest_gigapath_index = np.argmax(gigapath_means) # max(range(len(gigapath_means)), key=lambda i: gigapath_means[i])
    # pdb.set_trace()
    # # Add the highest indices to the significant indices
    # significant_indices.extend([highest_resnet50_index, highest_uni_index, highest_gigapath_index])

    # Create text annotations for significant points only
    texts = []
    for i in significant_indices:
        # resnet_label = f'{resnet50_means[i]:.2f} ± {resnet50_stds[i]:.2f}'
        uni_label = f'{uni_means[i]:.2f} ± {uni_stds[i]:.2f}'
        # gigapath_label = f'{gigapath_means[i]:.2f} ± {gigapath_stds[i]:.2f}'
        
        # texts.append(ax.text(i, resnet50_means[i], resnet_label, color=colors['resnet50'], ha='center', va='bottom', fontsize=13))
        texts.append(ax.text(i, uni_means[i], uni_label, color=colors['uni'], ha='center', va='bottom', fontsize=13))
        # texts.append(ax.text(i, gigapath_means[i], gigapath_label, color=colors['gigapath'], ha='center', va='bottom', fontsize=13))
    
    ### plot highest points
    # texts.append(ax.text(highest_resnet50_index, resnet50_means[highest_resnet50_index],  f'{resnet50_means[highest_resnet50_index]:.2f} ± {resnet50_stds[highest_resnet50_index]:.2f}', color=colors['resnet50'], ha='center', va='bottom', fontweight='bold', fontsize=13))
    texts.append(ax.text(highest_uni_index, uni_means[highest_uni_index], f'{uni_means[highest_uni_index]:.2f} ± {uni_stds[highest_uni_index]:.2f}', color=colors['uni'], ha='center', va='bottom', fontweight='bold', fontsize=13))
    # texts.append(ax.text(highest_gigapath_index, gigapath_means[highest_gigapath_index], f'{gigapath_means[highest_gigapath_index]:.2f} ± {gigapath_stds[highest_gigapath_index]:.2f}', color=colors['gigapath'], ha='center', va='bottom', fontweight='bold', fontsize=13))


    adjust_text(texts, arrowprops=dict(arrowstyle='->', color='k', lw=1.5))
    # Add labels for the mean values
    ax.legend()

    # Set the axis labels and title
    ax.set_xlabel('Epoch', fontsize=16)
    ax.set_ylabel('Avg. cross-validation score', fontsize=16)
    ax.set_title(f'{title}', fontsize=20)

    # Set the x-axis ticks
    ax.set_xticks(x)
    plt.xticks(fontsize=12)              # Increase x-tick label font size
    plt.yticks(fontsize=12)     
    ax.grid(True, linestyle='--', alpha=0.85)
    # Show the plot
    plt.show()

In [ ]:
# import matplotlib.pyplot as plt
# import numpy as np

# # Sample data: replace with your actual F1-scores and epochs
# models = ['Transformer', 'TransMIL', 'DTFD-MIL', 'CAMIL', 'CLAM', 'AttentionMIL', 'GraphTransformer']
# feature_encoders = ['ResNet50', 'UNI', 'Prov-Gigapath']
# colors = ['lightcoral', 'forestgreen', 'cyan']

# # Randomly generated F1-scores and epochs for demonstration; replace with your actual data
# # np.random.seed(0)
# f1_scores = {
#     models[0]: [
#         ([0.703, 0.691, 0.707, 0.672, 0.677], [5, 2, 4, 4, 2]),
#         ([0.690, 0.735, 0.673, 0.660, 0.684], [3, 6, 2, 2, 4]),
#         ([0.750, 0.716, 0.714, 0.642, 0.684], [7, 5, 6, 3, 4])
#     ],
#     models[1]: [
#         ([0.710, 0.695, 0.705, 0.675, 0.680], [4, 3, 5, 4, 3]),
#         ([0.695, 0.740, 0.670, 0.665, 0.690], [3, 6, 2, 2, 4]),
#         ([0.755, 0.720, 0.710, 0.645, 0.680], [7, 5, 6, 3, 4])
#     ],
#     models[2]: [
#         ([0.715, 0.700, 0.710, 0.680, 0.685], [4, 3, 5, 4, 3]),
#         ([0.700, 0.745, 0.675, 0.670, 0.695], [3, 6, 2, 2, 4]),
#         ([0.760, 0.725, 0.715, 0.650, 0.685], [7, 5, 6, 3, 4])
#     ],
#     models[3]: [
#         ([0.720, 0.705, 0.715, 0.685, 0.690], [4, 3, 5, 4, 3]),
#         ([0.705, 0.750, 0.680, 0.675, 0.700], [3, 6, 2, 2, 4]),
#         ([0.765, 0.730, 0.720, 0.655, 0.690], [7, 5, 6, 3, 4])
#     ],
#     models[4]: [
#         ([0.725, 0.710, 0.720, 0.690, 0.695], [4, 3, 5, 4, 3]),
#         ([0.710, 0.755, 0.685, 0.680, 0.705], [3, 6, 2, 2, 4]),
#         ([0.770, 0.735, 0.725, 0.660, 0.695], [7, 5, 6, 3, 4])
#     ],
#     models[5]: [
#         ([0.730, 0.715, 0.725, 0.695, 0.700], [4, 3, 5, 4, 3]),
#         ([0.715, 0.760, 0.690, 0.685, 0.710], [3, 6, 2, 2, 4]),
#         ([0.775, 0.740, 0.730, 0.665, 0.700], [7, 5, 6, 3, 4])
#     ],
#     models[6]: [
#         ([0.735, 0.720, 0.730, 0.700, 0.705], [4, 3, 5, 4, 3]),
#         ([0.720, 0.765, 0.695, 0.690, 0.715], [3, 6, 2, 2, 4]),
#         ([0.780, 0.745, 0.735, 0.670, 0.705], [7, 5, 6, 3, 4])
#     ]
# }

# # Prepare data for plotting
# data = []
# positions = []
# epoch_ranges = []
# mean_scores = []
# for i, model in enumerate(models):
#     for j, (scores, epochs) in enumerate(f1_scores[model]):
#         data.append(scores)
#         positions.append(i * (len(feature_encoders) + 1) + j + 1)
#         epoch_ranges.append((min(epochs), max(epochs)))
#         mean_scores.append(np.mean(scores))

# # Create grouped box plots
# fig, ax = plt.subplots(figsize=(15, 10))

# # Create the box plot
# boxplot = ax.boxplot(data, positions=positions, patch_artist=True, widths=0.42)

# # Color the box plots
# for patch, color in zip(boxplot['boxes'], colors * len(models)):
#     patch.set_facecolor(color)

# # Set the x-axis labels
# ax.set_xticks([i * (len(feature_encoders) + 1) + 2. for i in range(len(models))])
# ax.set_xticklabels(models)

# # Add legend
# for i, color in enumerate(colors):
#     ax.plot([], c=color, label=feature_encoders[i])
# ax.legend()

# # Annotate the epoch ranges
# for i, (min_epoch, max_epoch) in enumerate(epoch_ranges):
#     ax.annotate(f'({min_epoch}, {max_epoch})',
#                 xy=(positions[i], max(data[i]) + 0.002),
#                 ha='center')
#     # Annotate the mean score on the center line of the box plot
#     mean_y = mean_scores[i]
#     ax.annotate(f'{mean_y:.3f}',
#                 xy=(positions[i], mean_y),
#                 xytext=(0, 10),
#                 textcoords='offset points',
#                 ha='center',
#                 va='center',
#                 fontsize=9,
#                 color='black',
#                 weight='bold',
#                 bbox=dict(facecolor='white', alpha=0.6, edgecolor='none')
#                 )

# # Add grid for better separation between models
# ax.grid(True)
# # ax.set_axisbelow(True)

# # Labels and title
# ax.set_title('Best Cross-Validation F1-Scores per Feature Encoders Across MIL Algorithms')
# ax.set_xlabel('MIL Algorithms')
# ax.set_ylabel('F1-Score')

# plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_cross_validation_scores(f1_scores, 
                                 feature_encoders=['ResNet50', 'UNI', 'Prov-Gigapath'], 
                                 colors=['cyan', 'lavender', 'lightcoral'], 
                                 title='Best Cross-Validation F1-Scores per Feature Encoders Across MIL Algorithms', 
                                 xlabel='MIL Algorithms', 
                                 ylabel='F1-Score'):
    """
    Plot cross-validation scores for different models and feature encoders.
    
    Parameters:
    - f1_scores: dict, keys are model names, values are lists of tuples, 
                 where each tuple contains a list of best scores and a list of corresponding epochs.
    - feature_encoders: list, names of the feature encoders.
    - colors: list, colors for the box plots of each feature encoder.
    - title: str, title of the plot.
    - xlabel: str, label for the x-axis.
    - ylabel: str, label for the y-axis.
    """
    
    models = list(f1_scores.keys())
    
    # Prepare data for plotting
    data = []
    positions = []
    epoch_ranges = []
    mean_scores = []
    for i, model in enumerate(models):
        for j, (scores, epochs) in enumerate(f1_scores[model]):
            data.append(scores)
            positions.append(i * (len(feature_encoders) + 1) + j + 1)
            epoch_ranges.append((min(epochs), max(epochs)))
            mean_scores.append(np.mean(scores))

    # Create grouped box plots
    fig, ax = plt.subplots(figsize=(15, 10))

    # Create the box plot
    boxplot = ax.boxplot(data, positions=positions, patch_artist=True, widths=0.42,
                         medianprops=dict(color='black', linewidth=2))

    # Color the box plots
    for patch, color in zip(boxplot['boxes'], colors * len(models)):
        patch.set_facecolor(color)

    # Set the x-axis labels
    ax.set_xticks([i * (len(feature_encoders) + 1) + 2. for i in range(len(models))])
    ax.set_xticklabels(models)

    # Add legend
    for i, color in enumerate(colors):
        ax.plot([], c=color, label=feature_encoders[i])
    ax.legend()

    # Annotate the epoch ranges and mean scores
    for i, (min_epoch, max_epoch) in enumerate(epoch_ranges):
        ax.annotate(f'({min_epoch}, {max_epoch})',
                    xy=(positions[i], max(data[i]) + 0.002),
                    ha='center')
        # Annotate the mean score on the center line of the box plot
        mean_y = mean_scores[i]
        ax.annotate(f'{mean_y:.3f}',
                    xy=(positions[i], mean_y),
                    xytext=(0, 10),
                    textcoords='offset points',
                    ha='center',
                    va='center',
                    fontsize=9,
                    color='black',
                    weight='bold',
                    bbox=dict(facecolor='white', alpha=0.6, edgecolor='none'))

    # Add grid for better separation between models
    ax.grid(True)

    # Labels and title
    ax.set_title(title)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)

    plt.show()

# Example usage
models = ['Transformer', 'TransMIL', 'DTFD-MIL', 'CAMIL', 'CLAM', 'AttentionMIL', 'GraphTransformer']
models = ['Transformer', 'TransMIL', 'CAMIL', 'GraphTransformer']





In [ ]:
wandb.init(mode='disabled')

# transformer

In [ ]:
entity = 'cell-x'
project = 'g0-arrest-uni-cv5'
runs1 = wandb.Api().runs(f"{entity}/{project}")
entity = 'cell-x'
project = 'g0-arrest-resnet50-cv5'
runs2 =  wandb.Api().runs(f"{entity}/{project}")
entity = 'cell-x'
project = 'g0-arrest-gigapath-cv5'
runs3 =  wandb.Api().runs(f"{entity}/{project}")


In [ ]:
uni_f1_scores, uni_aurocval_scores = get_val_metrics(runs=runs1, index=8)
resnet50_f1_scores, resnet50_aurocval_scores = get_val_metrics(runs=runs2, index=8)
gigapath_f1_scores, gigapath_aurocval_scores = get_val_metrics(runs=runs3, index=8)


In [ ]:
# transformer_resnet50_f1, transformer_resnet50_f1_std, transformer_resnet50_epoch = get_mean_std_per_agg(resnet50_f1_scores)
# transformer_uni_f1, transformer_uni_f1_std, transformer_uni_epoch = get_mean_std_per_agg(uni_f1_scores)
# transformer_gigapath_f1, transformer_gigapath_f1_std, transformer_gigapath_epoch = get_mean_std_per_agg(gigapath_f1_scores)

transformer_resnet50_f1, transformer_resnet50_f1_epoch = get_max(resnet50_f1_scores)
transformer_uni_f1, transformer_uni_f1_epoch = get_max(uni_f1_scores)
transformer_gigapath_f1, transformer_gigapath_f1_epoch = get_max(gigapath_f1_scores)


In [ ]:
transformer_resnet50_auroc, transformer_resnet50_auroc_epoch = get_max(resnet50_aurocval_scores)
transformer_uni_auroc, transformer_uni_auroc_epoch = get_max(uni_aurocval_scores)
transformer_gigapath_auroc, transformer_gigapath_auroc_epoch = get_max(gigapath_aurocval_scores)

# auroc_uni_means, auroc_uni_stds = get_max(uni_aurocval_scores)
# auroc_resnet50_means, auroc_resnet50_stds = get_mean_std(resnet50_aurocval_scores)
# auroc_gigapath_means, auroc_gigapath_stds = get_mean_std(gigapath_aurocval_scores)

# TransMIL

In [ ]:
entity = 'cell-x'
project = 'g0-arrest-resnet50-transmil-cv5'
runs1 = wandb.Api().runs(f"{entity}/{project}")
entity = 'cell-x'
project = 'g0-arrest-uni-transmil-cv5'
runs2 =  wandb.Api().runs(f"{entity}/{project}")
entity = 'cell-x'
project = 'g0-arrest-gigapath-transmil-cv5'
runs3 =  wandb.Api().runs(f"{entity}/{project}")


In [ ]:
resnet50_f1_scores_transmil, resnet50_aurocval_scores_transmil = get_val_metrics(runs=runs1, index=31)
uni_f1_scores_transmil, uni_aurocval_scores_transmil = get_val_metrics(runs=runs2, index=31)
gigapath_f1_scores_transmil, gigapath_aurocval_scores_transmil = get_val_metrics(runs=runs3, index=31)


In [ ]:

# resnet50_means_transmil, resnet50_stds_transmil = get_mean_std(resnet50_f1_scores_transmil)
# uni_means_transmil, uni_stds_transmil = get_mean_std(uni_f1_scores_transmil)
# gigapath_means_transmil, gigapath_stds_transmil = get_mean_std(gigapath_f1_scores_transmil)

transmil_resnet50_f1, transmil_resnet50_f1_epoch = get_max(resnet50_f1_scores_transmil)
transmil_uni_f1, transmil_uni_f1_epoch = get_max(uni_f1_scores_transmil)
transmil_gigapath_f1, transmil_gigapath_f1_epoch = get_max(gigapath_f1_scores_transmil)

transmil_resnet50_auroc, transmil_resnet50_auroc_epoch = get_max(resnet50_aurocval_scores_transmil)
transmil_uni_auroc, transmil_uni_auroc_epoch = get_max(uni_aurocval_scores_transmil)
transmil_gigapath_auroc, transmil_gigapath_auroc_epoch = get_max(gigapath_aurocval_scores_transmil)



# CAMIL
### 30 epochs

In [ ]:
entity = 'anwangxuelong'
project1 = 'g0-arrest-resnet50-cv5-camil' 
project2 = 'g0-arrest-uni-cv5-camil'
project3 = 'g0-arrest-gigapath-cv5-camil'



runs1 = wandb.Api().runs(f"{entity}/{project1}") # check what happened, why are there 6 runs
runs2 =  wandb.Api().runs(f"{entity}/{project2}")
runs3 =  wandb.Api().runs(f"{entity}/{project3}")

In [ ]:
resnet50_f1_scores, resnet50_aurocval_scores = get_val_metrics(runs=runs1, index=1)


### joining cv scores due to resuming from checkpoint and deleting overlapping epochs

In [ ]:
def update_scores(scores, index_left, index_right, index_eliminate):
    
    
    resumed_scores = scores[index_left][:-index_eliminate] + scores[index_right]  # Assuming new_values is the second list

    # Replace the second list with the new F1 score
    scores[index_right] = resumed_scores

    # Remove the third list
    del scores[index_left]

    return scores

In [ ]:
[len(sublist) for sublist in resnet50_f1_scores]
# resnet50_f1_scores[3]

In [ ]:
# resnet50_f1_score_cv1_resumed = resnet50_f1_scores[3][:-5] + resnet50_f1_scores[4] # resumed from epoch 5
# resnet50_f1_scores[1] = resnet50_f1_score_cv1_resumed
# del resnet50_f1_scores[2]
resnet50_f1_scores = update_scores(resnet50_f1_scores, index_left=3, index_right=4, index_eliminate=5)

In [ ]:
# resnet50_aurocval_scores = update_scores(resnet50_aurocval_scores, index_left=2, index_right=1, index_eliminate=5)
resnet50_aurocval_scores = update_scores(resnet50_aurocval_scores, index_left=3, index_right=4, index_eliminate=5)

In [ ]:
[len(sublist) for sublist in resnet50_aurocval_scores]
# resnet50_f1_scores[4]

In [ ]:
uni_f1_scores, uni_aurocval_scores = get_val_metrics(runs=runs2, index=30)
gigapath_f1_scores, gigapath_aurocval_scores = get_val_metrics(runs=runs3, index=30)


In [ ]:
camil_resnet50_f1, camil_resnet50_f1_epoch = get_max(resnet50_f1_scores)
camil_uni_f1, camil_uni_f1_epoch = get_max(uni_f1_scores)
camil_gigapath_f1, camil_gigapath_f1_epoch = get_max(gigapath_f1_scores)

camil_resnet50_auroc, camil_resnet50_auroc_epoch = get_max(resnet50_aurocval_scores)
camil_uni_auroc, camil_uni_auroc_epoch = get_max(uni_aurocval_scores)
camil_gigapath_auroc, camil_gigapath_auroc_epoch = get_max(gigapath_aurocval_scores)



In [ ]:
# plot_results(uni_res = [uni_means, uni_stds], resnet_res = [resnet50_means, resnet50_stds], gigapath_res = [gigapath_means, gigapath_stds], title = 'CAMIL aggregator per feature encoder: \n Average 5-fold cross-validation F1-score')
plot_results_no_gigapath(uni_res = [uni_means, uni_stds], resnet_res = [resnet50_means, resnet50_stds], gigapath_res = [gigapath_means, gigapath_stds], title = 'CAMIL aggregator per feature encoder: \n Average 5-fold cross-validation F1-score')


In [ ]:
auroc_resnet50_means, auroc_resnet50_stds = get_mean_std(resnet50_aurocval_scores)
auroc_uni_means, auroc_uni_stds = get_mean_std(uni_aurocval_scores)
auroc_gigapath_means, auroc_gigapath_stds = get_mean_std(gigapath_aurocval_scores)

In [ ]:
# plot_results(uni_res = [auroc_uni_means, auroc_uni_stds], resnet_res = [auroc_resnet50_means, auroc_resnet50_stds], gigapath_res = [auroc_gigapath_means, auroc_gigapath_stds], title = 'CAMIL aggregator per feature encoder: \n Average 5-fold cross-validation AUROC score')
plot_results_no_gigapath(uni_res = [auroc_uni_means, auroc_uni_stds], resnet_res = [auroc_resnet50_means, auroc_resnet50_stds], gigapath_res = [auroc_gigapath_means, auroc_gigapath_stds], title = 'CAMIL aggregator per feature encoder: \n Average 5-fold cross-validation AUROC score')


# DTFD-MIL
### 42 epochs

In [ ]:
entity = 'anwangxuelong'
project1 = 'g0-arrest-resnet50-dtfdmil-cv5-multisteplr' 
project2 = 'g0-arrest-uni-dtfdmil-cv5-multisteplr'
project3 = 'g0-arrest-gigapath-dtfdmil-cv5-multisteplr'



runs1 = wandb.Api().runs(f"{entity}/{project1}") # check what happened, why are there 6 runs
runs2 =  wandb.Api().runs(f"{entity}/{project2}")
runs3 =  wandb.Api().runs(f"{entity}/{project3}")

In [ ]:
resnet50_f1_scores, resnet50_aurocval_scores = get_val_metrics_scan(runs=runs1)


In [ ]:
[len(sublist) for sublist in resnet50_f1_scores]


In [ ]:
[len(sublist) for sublist in resnet50_f1_scores]
# len(resnet50_f1_scores[1][:-3] + resnet50_f1_scores[2])
len(resnet50_f1_scores[3][:-1] + resnet50_f1_scores[4])

resnet50_f1_scores = update_scores(resnet50_f1_scores, index_left=1, index_right=2, index_eliminate=3)
resnet50_aurocval_scores = update_scores(resnet50_aurocval_scores, index_left=1, index_right=2, index_eliminate=3)

# uni_f1_scores = update_scores(uni_f1_scores, index_left=2, index_right=3, index_eliminate=10)
# uni_aurocval_scores = update_scores(uni_aurocval_scores, index_left=2, index_right=3, index_eliminate=10)


In [ ]:
[len(sublist) for sublist in resnet50_f1_scores]
resnet50_f1_scores = update_scores(resnet50_f1_scores, index_left=2, index_right=3, index_eliminate=1)
resnet50_aurocval_scores = update_scores(resnet50_aurocval_scores, index_left=2, index_right=3, index_eliminate=1)


In [ ]:
[len(sublist) for sublist in resnet50_f1_scores]


In [ ]:
uni_f1_scores, uni_aurocval_scores = get_val_metrics_scan(runs=runs2)
gigapath_f1_scores, gigapath_aurocval_scores = get_val_metrics_scan(runs=runs3)


# uni_means, uni_stds = get_mean_std(uni_f1_scores)
# gigapath_means, gigapath_stds = get_mean_std(gigapath_f1_scores)

In [ ]:
[len(sublist) for sublist in uni_aurocval_scores]
# len(uni_aurocval_scores[2][:-10] + uni_aurocval_scores[3])


In [ ]:
# uni_aurocval_scores[3]
# uni_f1_scores[2][:-10] + uni_aurocval_scores[3]
uni_f1_scores = update_scores(uni_f1_scores, index_left=2, index_right=3, index_eliminate=10)
uni_aurocval_scores = update_scores(uni_aurocval_scores, index_left=2, index_right=3, index_eliminate=10)


In [ ]:
[len(sublist) for sublist in gigapath_f1_scores]
len(gigapath_f1_scores[2][:-1] + gigapath_f1_scores[3])
# gigapath_f1_scores[3]
gigapath_f1_scores = update_scores(gigapath_f1_scores, index_left=2, index_right=3, index_eliminate=1)
gigapath_aurocval_scores = update_scores(gigapath_aurocval_scores, index_left=2, index_right=3, index_eliminate=1)



In [ ]:
[len(sublist) for sublist in gigapath_aurocval_scores]
[len(sublist) for sublist in uni_aurocval_scores]

In [ ]:

resnet50_means, resnet50_stds = get_mean_std(resnet50_f1_scores)
uni_means, uni_stds = get_mean_std(uni_f1_scores)
gigapath_means, gigapath_stds = get_mean_std(gigapath_f1_scores)

In [ ]:
# plot_results(resnet_res = [resnet50_means, resnet50_stds], \
#             uni_res = [uni_means, uni_stds], \
#             gigapath_res = [gigapath_means, gigapath_stds], title = 'DTFD-MIL aggregator per feature encoder: \n Average 5-fold cross-validation F1-score')
plot_results_no_gigapath(resnet_res = [resnet50_means, resnet50_stds], \
            uni_res = [uni_means, uni_stds], \
            gigapath_res = [gigapath_means, gigapath_stds], title = 'DTFD-MIL aggregator per feature encoder: \n Average 5-fold cross-validation F1-score')


In [ ]:

resnet50_means, resnet50_stds = get_mean_std(resnet50_aurocval_scores)
uni_means, uni_stds = get_mean_std(uni_aurocval_scores)
gigapath_means, gigapath_stds = get_mean_std(gigapath_aurocval_scores)

In [ ]:
plot_results(resnet_res = [resnet50_means, resnet50_stds], \
            uni_res = [uni_means, uni_stds], \
            gigapath_res = [gigapath_means, gigapath_stds], title = 'DTFD-MIL aggregator per feature encoder: \n Average 5-fold cross-validation AUROC score')


# AttentionMIL

In [ ]:
entity1 = 'cell-x'
entity2 = 'anwangxuelong'
project1 = 'g0-arrest-resnet50-attentionMIL_cv5' 
project2 = 'g0-arrest-uni-attentionMIL_cv5'
project3 = 'g0-arrest-gigapath-attentionMIL_cv5'



runs1 = wandb.Api().runs(f"{entity1}/{project1}") # check what happened, why are there 6 runs
runs2 =  wandb.Api().runs(f"{entity1}/{project2}")
runs3 =  wandb.Api().runs(f"{entity2}/{project3}")

In [ ]:
resnet50_f1_scores, resnet50_aurocval_scores = get_val_metrics(runs=runs1)
uni_f1_scores, uni_aurocval_scores = get_val_metrics(runs=runs2)
gigapath_f1_scores, gigapath_aurocval_scores = get_val_metrics(runs=runs3)


# uni_means, uni_stds = get_mean_std(uni_f1_scores)
# gigapath_means, gigapath_stds = get_mean_std(gigapath_f1_scores)

In [ ]:
[len(sublist) for sublist in resnet50_f1_scores]


In [ ]:
[len(sublist) for sublist in uni_f1_scores]
# resnet50_f1_scores = update_scores(resnet50_f1_scores, index_left=4, index_right=3, index_eliminate=5)
# uni_f1_scores[3]
# uni_f1_scores[4]
uni_f1_scores = update_scores(uni_f1_scores, index_left=3, index_right=4, index_eliminate=1)
uni_aurocval_scores = update_scores(uni_aurocval_scores, index_left=3, index_right=4, index_eliminate=1)


In [ ]:
[len(sublist) for sublist in uni_f1_scores]


In [ ]:
[len(sublist) for sublist in gigapath_f1_scores]
gigapath_f1_scores = update_scores(gigapath_f1_scores, index_left=3, index_right=4, index_eliminate=4)
gigapath_f1_scores = update_scores(gigapath_f1_scores, index_left=4, index_right=5, index_eliminate=2)

In [ ]:
gigapath_aurocval_scores = update_scores(gigapath_aurocval_scores, index_left=3, index_right=4, index_eliminate=4)
gigapath_aurocval_scores = update_scores(gigapath_aurocval_scores, index_left=4, index_right=5, index_eliminate=2)

In [ ]:
[len(sublist) for sublist in gigapath_aurocval_scores]


In [ ]:
resnet50_means, resnet50_stds = get_mean_std(resnet50_f1_scores)
uni_means, uni_stds = get_mean_std(uni_f1_scores)
gigapath_means, gigapath_stds = get_mean_std(gigapath_f1_scores)

In [ ]:
# plot_results(resnet_res = [resnet50_means,resnet50_stds], \
#             uni_res = [uni_means, uni_stds], \
#             gigapath_res = [gigapath_means, gigapath_stds], title = 'AttentionMIL aggregator per feature encoder: \n Average 5-fold cross-validation F1-score')
plot_results_no_gigapath(resnet_res = [resnet50_means,resnet50_stds], \
            uni_res = [uni_means, uni_stds], \
            gigapath_res = [gigapath_means, gigapath_stds], title = 'AttentionMIL aggregator per feature encoder: \n Average 5-fold cross-validation F1-score')


In [ ]:
resnet50_means, resnet50_stds = get_mean_std(resnet50_aurocval_scores)
uni_means, uni_stds = get_mean_std(uni_aurocval_scores)
gigapath_means, gigapath_stds = get_mean_std(gigapath_aurocval_scores)

In [ ]:
# plot_results(uni_res = [uni_means, uni_stds], resnet_res = [resnet50_means, resnet50_stds], gigapath_res = [gigapath_means, gigapath_stds], title = 'AttentionMIL aggregator per feature encoder: \n Average 5-fold cross-validation AUROC score')
plot_results_no_gigapath(uni_res = [uni_means, uni_stds], resnet_res = [resnet50_means, resnet50_stds], gigapath_res = [gigapath_means, gigapath_stds], title = 'AttentionMIL aggregator per feature encoder: \n Average 5-fold cross-validation AUROC score')


# GraphTransformer

In [ ]:
entity = 'anwangxuelong'
project1 = 'g0-arrest-resnet50-cv5-graph-transformer' 
project2 = 'g0-arrest-uni-cv5-graph-transformer'
project3 = 'g0-arrest-gigapath-cv5-graph-transformer'



runs1 = wandb.Api().runs(f"{entity}/{project1}") # check what happened, why are there 6 runs
runs2 =  wandb.Api().runs(f"{entity}/{project2}")
runs3 =  wandb.Api().runs(f"{entity}/{project3}")

In [ ]:
resnet50_f1_scores, resnet50_aurocval_scores = get_val_metrics(runs=runs1)
uni_f1_scores, uni_aurocval_scores = get_val_metrics(runs=runs2)
gigapath_f1_scores, gigapath_aurocval_scores = get_val_metrics(runs=runs3)


In [ ]:
[len(sublist) for sublist in resnet50_f1_scores]
resnet50_f1_scores = update_scores(resnet50_f1_scores, index_left=1, index_right=2, index_eliminate=6)
resnet50_aurocval_scores = update_scores(resnet50_aurocval_scores, index_left=1, index_right=2, index_eliminate=6)



In [ ]:
[len(sublist) for sublist in resnet50_f1_scores]


In [ ]:
[len(sublist) for sublist in resnet50_f1_scores]
[len(sublist) for sublist in uni_aurocval_scores]
uni_f1_scores = update_scores(uni_f1_scores, index_left=1, index_right=2, index_eliminate=30)
uni_aurocval_scores = update_scores(uni_aurocval_scores, index_left=1, index_right=2, index_eliminate=30)


In [ ]:
[len(sublist) for sublist in uni_aurocval_scores]


In [ ]:
[len(sublist) for sublist in uni_aurocval_scores]
[len(sublist) for sublist in gigapath_f1_scores]
gigapath_f1_scores = update_scores(gigapath_f1_scores, index_left=1, index_right=2, index_eliminate=12)
gigapath_aurocval_scores = update_scores(gigapath_aurocval_scores,index_left=1, index_right=2, index_eliminate=12)


In [ ]:
[len(sublist) for sublist in gigapath_f1_scores]
gigapath_f1_scores = update_scores(gigapath_f1_scores, index_left=4, index_right=5, index_eliminate=6)
gigapath_aurocval_scores = update_scores(gigapath_aurocval_scores, index_left=4, index_right=5, index_eliminate=6)


In [ ]:
[len(sublist) for sublist in gigapath_f1_scores]


In [ ]:

graphtransformer_resnet50_f1, graphtransformer_resnet50_f1_epoch = get_max(resnet50_f1_scores)
graphtransformer_uni_f1, graphtransformer_uni_f1_epoch = get_max(uni_f1_scores)
graphtransformer_gigapath_f1, graphtransformer_gigapath_f1_epoch = get_max(gigapath_f1_scores)

graphtransformer_resnet50_auroc, graphtransformer_resnet50_auroc_epoch = get_max(resnet50_aurocval_scores)
graphtransformer_uni_auroc, graphtransformer_uni_auroc_epoch = get_max(uni_aurocval_scores)
graphtransformer_gigapath_auroc, graphtransformer_gigapath_auroc_epoch = get_max(gigapath_aurocval_scores)



# Multimodal TransMIL

In [ ]:
entity = 'anwangxuelong'
# project1 = 'g0-arrest-resnet50-cv5-graph-transformer' 
project2 = 'g0-arrest-uni-transmil-multimodal-cv5'
# project3 = 'g0-arrest-gigapath-cv5-graph-transformer'



# runs1 = wandb.Api().runs(f"{entity}/{project1}") # check what happened, why are there 6 runs
runs2 =  wandb.Api().runs(f"{entity}/{project2}")
# runs3 =  wandb.Api().runs(f"{entity}/{project3}")

In [ ]:
uni_f1_scores, uni_aurocval_scores = get_val_metrics(runs=runs2)


In [ ]:
[len(sublist) for sublist in uni_f1_scores]


In [ ]:
uni_means, uni_stds = get_mean_std(uni_f1_scores)


In [ ]:
plot_results_only_uni(resnet_res = [uni_means, uni_stds], \
            uni_res = [uni_means, uni_stds], \
            gigapath_res = [uni_means, uni_stds], title = 'TransMIL Multimodal Classification with UNI: \n Average 5-fold cross-validation F1-score')


In [ ]:
uni_means, uni_stds = get_mean_std(uni_aurocval_scores)


In [ ]:
plot_results_only_uni(resnet_res = [uni_means, uni_stds], \
            uni_res = [uni_means, uni_stds], \
            gigapath_res = [uni_means, uni_stds], title = 'TransMIL Multimodal Classification with UNI: \n Average 5-fold cross-validation AUROC-score')


# Combining DTFD-MIL-TransMIL

In [ ]:
entity = 'anwangxuelong'
# project1 = 'g0-arrest-resnet50-cv5-graph-transformer' 
project2 = 'g0-arrest-uni-dtfdtransmil-cv5'
# project3 = 'g0-arrest-gigapath-cv5-graph-transformer'



# runs1 = wandb.Api().runs(f"{entity}/{project1}") # check what happened, why are there 6 runs
runs2 =  wandb.Api().runs(f"{entity}/{project2}")
# runs3 =  wandb.Api().runs(f"{entity}/{project3}")

In [ ]:
uni_f1_scores, uni_aurocval_scores = get_val_metrics_scan(runs=runs2)


In [ ]:
[len(sublist) for sublist in uni_f1_scores]
uni_f1_scores = update_scores(uni_f1_scores, index_left=4, index_right=5, index_eliminate=7)
uni_aurocval_scores = update_scores(uni_aurocval_scores, index_left=4, index_right=5, index_eliminate=7)



In [ ]:
[len(sublist) for sublist in uni_f1_scores]


In [ ]:
uni_means, uni_stds = get_mean_std(uni_f1_scores)


In [ ]:
plot_results_only_uni(resnet_res = [uni_means, uni_stds], \
            uni_res = [uni_means, uni_stds], \
            gigapath_res = [uni_means, uni_stds], title = 'DTFD-TransMIL Classification with UNI: \n Average 5-fold cross-validation F1-score')


In [ ]:
uni_means, uni_stds = get_mean_std(uni_aurocval_scores)


In [ ]:
plot_results_only_uni(resnet_res = [uni_means, uni_stds], \
            uni_res = [uni_means, uni_stds], \
            gigapath_res = [uni_means, uni_stds], title = 'DTFD-TransMIL Classification with UNI: \n Average 5-fold cross-validation AUROC-score')


In [ ]:
auroc_scores = {
    models[0]: [
        (transformer_resnet50_auroc, transformer_resnet50_auroc_epoch),
        (transformer_uni_auroc, transformer_uni_auroc_epoch ),
        (transformer_gigapath_auroc, transformer_gigapath_auroc_epoch)
    ],
    models[1]: [
        (transmil_resnet50_auroc, transmil_resnet50_auroc_epoch),
        (transmil_uni_auroc, transmil_uni_auroc_epoch),
        (transmil_gigapath_auroc, transmil_gigapath_auroc_epoch)
    ],
    models[2]: [
        (camil_resnet50_auroc, camil_resnet50_auroc_epoch),
        (camil_uni_auroc, camil_uni_auroc_epoch),
        (camil_gigapath_auroc, camil_gigapath_auroc_epoch)
    ],
    models[3]:[
        (graphtransformer_resnet50_auroc, graphtransformer_resnet50_auroc_epoch),
        (graphtransformer_uni_auroc, graphtransformer_uni_auroc_epoch),
        (graphtransformer_gigapath_auroc, graphtransformer_gigapath_auroc_epoch),
    ]

}

plot_cross_validation_scores(auroc_scores,
                             title='Best Cross-Validation AUROC-Scores per Feature Encoder Across MIL Algorithms',
                             ylabel='AUROC Score')

In [ ]:
wandb.finish()